# Проект: Выбор локации для скважины

## Описание проекта
<b> Нужно решить, где бурить новую скважину. Шаги для выбора локации обычно такие:
Шаги для выбора локации обычно такие:
    
* В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
* Строят модель для предсказания объёма запасов в новых скважинах;
* Выбирают скважины с самыми высокими оценками значений;
* Определяют регион с максимальной суммарной прибылью отобранных скважин.
    
<b> Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

## Вывод:

<div class="alert alert-block alert-info">
Отсутствие прогнозируемых убытков наблюдаю только во втором регионе, что видно по значениям 2,5% квантиля. Левая граница, которая отделяет 2.5 % данных для 1-го и 3-го регионов принимает отрицательное значение, что говорит о том, что более 2.5 % выборок имеют отрицательную прибыль. По заданию исследования нужно оставить лишь те регионы, в которых вероятность убытков меньше 2,5%, это 2-ой регион. В связи с этим для дальнейшей работы рекомендуется оставить только второй регион.
    </div>

## Введение и описание данных

Условия задачи: Для обучения модели нам подходит только линейная регрессия (остальные — недостаточно предсказуемые). При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки. Бюджет на разработку скважин в регионе — 10 млрд рублей. При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей. После оценки рисков оставим лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выберем регион с наибольшей средней прибылью. Данные синтетические: детали контрактов и характеристики месторождений не разглашаются. Значения столбцов: id — уникальный идентификатор скважины; f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы); product — объём запасов в скважине (тыс. баррелей).

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from joblib import dump
from sklearn.linear_model import LogisticRegression 
import numpy as np
from scipy import stats as st
import warnings
warnings.filterwarnings("ignore")

geo_data_0 = pd.read_csv('geo_data_0.csv')
geo_data_1 = pd.read_csv('geo_data_1.csv')
geo_data_2 = pd.read_csv('geo_data_2.csv')


print(geo_data_0.head()) 

print(geo_data_1.head()) 

print(geo_data_2.head()) 


      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


<b> Поиск пропущенных значений в наборе данных

In [3]:
geo_data_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [4]:
geo_data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [5]:
geo_data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

<b> Поиск дубликатов

In [6]:
geo_data_0.duplicated().sum()

0

In [7]:
geo_data_1.duplicated().sum()

0

In [8]:
geo_data_2.duplicated().sum()

0

In [9]:
# В столбце ID нет смысла и выглядит запутывающи, удаляю его.
geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

In [10]:
print(geo_data_0.describe())

print(geo_data_1.describe()) 

print(geo_data_2.describe()) 

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

## Обучение и проверка модели

Разбиваю данные на выборки: Обучающую, валидационную, тестовую в соотношении 75-25 соотвественно. 
Так как у признаков разный масштаб, то стандартизирую их! Импортирую StandardScaler из модуля sklearn.preprocessing. 
Далее помещаю все это в функцию.

In [11]:
def i(geo_data):
   
    target = geo_data['product']
    features = geo_data.drop('product', axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    
    numeric = ['f0', 'f1', 'f2']
# Создаю объект структуры StandardScaler() и настраиваю его на обучающих данных. 
# Настройка — это вычисление среднего и дисперсии.    
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])
# Преобразую обучающую и валидационную выборки функцией transform(). Изменённые наборы сохраняю в переменных
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train, features_valid, target_train, target_valid

features_train_0, features_valid_0, target_train_0, target_valid_0 = i(geo_data_0) 
features_train_1, features_valid_1, target_train_1, target_valid_1 = i(geo_data_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = i(geo_data_2)

print(features_train_0.shape,  target_train_0.shape)
print(features_valid_0.shape, target_valid_0.shape) 

print(features_train_1.shape,  target_train_1.shape)
print(features_valid_1.shape, target_valid_1.shape) 

print(features_train_2.shape,  target_train_2.shape)
print(features_valid_2.shape, target_valid_2.shape) 

(75000, 3) (75000,)
(25000, 3) (25000,)
(75000, 3) (75000,)
(25000, 3) (25000,)
(75000, 3) (75000,)
(25000, 3) (25000,)


Обучаю модель и сделаю предсказания на валидационной выборке. Для обучения модели подходит только линейная регрессия(остальные недостаточно предсказуемые). Сохраню предсказания и ответы по каждому региону и выведу средний запас и RMSE модели по каждому региону. Корень из среднеквадратичной ошибки (Root Mean Squared Error)-Для того чтобы модель линейной регрессии можно было применять на практике необходимо сначала оценить её качество. 

In [12]:
model = LinearRegression()
model.fit(features_train_0, target_train_0)
predictions_valid_0 = pd.Series(model.predict(features_valid_0))
mse = mean_squared_error(target_valid_0, predictions_valid_0)
RMSE_0=mse**0.5
mean_predictions_valid_0 =predictions_valid_0.mean()

print("Linear Regression 0")
print("Mean =", mean_predictions_valid_0)
print("RMSE =", RMSE_0)

Linear Regression 0
Mean = 92.59256778438005
RMSE = 37.5794217150813


In [13]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predictions_valid_1 = pd.Series(model.predict(features_valid_1))
mse = mean_squared_error(target_valid_1, predictions_valid_1)
RMSE_1=mse**0.5
mean_predictions_valid_1 =predictions_valid_1.mean()

print("Linear Regression 1")
print("Mean =", mean_predictions_valid_1)
print("RMSE =", RMSE_1)

Linear Regression 1
Mean = 68.7285468954458
RMSE = 0.8930992867756168


In [14]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predictions_valid_2 = pd.Series(model.predict(features_valid_2))
mse = mean_squared_error(target_valid_2, predictions_valid_2)
RMSE_2=mse**0.5
mean_predictions_valid_2 =predictions_valid_2.mean()

print("Linear Regression 2")
print("Mean =", mean_predictions_valid_2)
print("RMSE =", RMSE_2)

Linear Regression 2
Mean = 94.96504596800506
RMSE = 40.02970873393434


<div class="alert alert-block alert-info">
<b> Чем ниже RMSE, тем лучше данная модель может «соответствовать» набору данных. Вижу, что самый низкий показатель RMSE у второй модели.
    </div>

## Подготовка к расчёту прибыли 

In [15]:
### Сохраняю все ключевые значения для расчетов в отдельных переменных
research_point = 500  # количество точек исследования
learning_point = 200 # количество точек для обучения
budget = 10000 # бюджет на разработку скважин млн.р.
bootstrap_sample = 1000 # количество выборок по заданию
income_per_unit = 0.45 # доход с каждой единицы продукта млн.р
probability_losses = 0.025 # вероятность убытков не меньше 2,5%

Рассчёт достаточного объема сырья для безубыточной разработки новой скважины. Сначала рассчитываю бюджет на одну скважину

In [16]:
budget_for_well = budget/learning_point
print('Бюджет на разработку 1 скважины в млн.р',budget_for_well)

Бюджет на разработку 1 скважины в млн.р 50.0


<b> Если разделить бюджет одной скважины на доход с каждой единицы продукта, то получу сколько баррелей должна давать скважина.

In [18]:
minimum_well_income = budget_for_well/income_per_unit
print('чтобы быть доходной скважина должна двавать баррелей', minimum_well_income )

чтобы быть доходной скважина должна двавать баррелей 111.11111111111111


В выбранном регионе № 2 среднее значение 68, что намного меньше. Интересно, сможет ли вообще в этом регионе окупиться добыча. Первый и второй регион по средней добыче на скважину немного выше, но все равно не дотягивают до нужного значения.

Напишу функцию для расчета прибыли по выбранным скважинам и предсказаниям модели. Сначала выберу скважины с 200 максимальными значениями предсказаний, просуммируем целевое значение объема сырья и рассчитаем прибыль для полученного объема сырья.

In [19]:
def revenue(target_valid, predictions_valid):
    probs_sorted = predictions_valid.sort_values(ascending=False).head(200)
    selected = target_valid.reset_index(drop=True)[probs_sorted.index]
    return (selected.sum() * income_per_unit-budget)


print('Регион 1, млн. руб.', revenue(target_valid_0, predictions_valid_0))
print('Регион 2, млн. руб.', revenue(target_valid_1, predictions_valid_1))
print('Регион 3, млн. руб.', revenue(target_valid_2, predictions_valid_2))

Регион 1, млн. руб. 3320.8260431398503
Регион 2, млн. руб. 2415.086696681512
Регион 3, млн. руб. 2710.3499635998323


<div class="alert alert-block alert-info">
<b> Рассчитана прибыль, если все скважины в каждом регионе разраватывать и выбирать 200 самых прибыльных. В реальности на разработку всех скважин не хватит бюджета. Так что далее по заданию беру 500 точек разработки и выбераю из них 200 самых прибыльных и смотрю как изменится прибыль после этого.
    </div>

## Расчет прибыли и рисков для каждого региона

Меняю функцию по расчету прибыли и беру значения по количеству точек исследования 500, количество точек для обучения 200

In [20]:
def revenue(target_valid, predictions_valid, learning_point):
    probs_sorted = predictions_valid.sort_values(ascending=False)
    selected = target_valid[probs_sorted.index][:learning_point]
    return (selected.sum() * income_per_unit-budget)

Использую технику bootstrap с 1000 выборок и нахожу распределение прибыли

In [21]:
def bootstrap(target_valid, predictions_valid,bootstrap_sample, research_point ): 
    state = np.random.RandomState(12345)   
    values = []
  
    for i in range(bootstrap_sample):
        target_subsample = target_valid.reset_index(drop=True).sample(research_point, replace=True, random_state=state)
        probs_subsample = predictions_valid[target_subsample.index] 
        rev = revenue(target_subsample, probs_subsample,learning_point)
        values.append(rev)

    

    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    risk = display(pd.Series(values))

    mean = values.mean()
    
    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("97.5%-квантиль:", upper)
   
    print('Риск убытков = {:.2%} '.format((pd.Series(values)<0).mean()))
  
    return (lower, upper,  mean, risk)


In [22]:
display(pd.Series([-2, -1, 4, 5]))
'Риск убытков = {:.2%} '.format((pd.Series([-2, -1, 4, 5])<0).mean())

0   -2
1   -1
2    4
3    5
dtype: int64

'Риск убытков = 50.00% '

Рассматриваю среднюю прибыль, 95% доверительный интервал и возможный риск убытков по каждому региону и выбераю наиболее подходящий.

In [23]:
bootstrap(target_valid_0, predictions_valid_0,bootstrap_sample, research_point)
print('Регион 1')


0      585.088996
1      536.393431
2      211.079359
3      265.280252
4      271.992922
          ...    
995    474.095510
996    855.997177
997    785.994755
998    354.221504
999    198.820277
Length: 1000, dtype: float64

Средняя выручка: 425.9385269105923
2.5%-квантиль: -102.09009483793592
97.5%-квантиль: 947.976353358369
Риск убытков = 6.00% 
Регион 1


In [24]:
bootstrap(target_valid_1, predictions_valid_1,bootstrap_sample, research_point)
print('Регион 2')

0      299.359372
1      382.831548
2      303.651161
3      658.190210
4      358.573613
          ...    
995    755.968347
996    766.728918
997    358.511413
998    509.843607
999    174.524307
Length: 1000, dtype: float64

Средняя выручка: 515.2227734432905
2.5%-квантиль: 68.87322537050254
97.5%-квантиль: 931.547591257049
Риск убытков = 1.00% 
Регион 2


In [25]:
bootstrap(target_valid_2, predictions_valid_2,bootstrap_sample, research_point)
print('Регион 3')

0      -95.044177
1      728.210597
2      611.217813
3      582.754149
4       25.014579
          ...    
995    566.865968
996    -11.648065
997    645.595218
998    536.735982
999    121.270210
Length: 1000, dtype: float64

Средняя выручка: 435.0083627827561
2.5%-квантиль: -128.88054732978893
97.5%-квантиль: 969.7069541802657
Риск убытков = 6.40% 
Регион 3


## Вывод

<div class="alert alert-block alert-info">
<b> По всем трем регионам показатели средней выручки сильно уменьшились по сравнению с предыдущим расчетом. Согласно расчетам показатели средней выручки во всех трех регионах подходят для дальнейших разработок, так как минимальная граница прибыльности сильно ниже. чем прогнозируемая прибыль по всем трем регионам. Отсутствие прогнозируемых убытков видно только во втором регионе,  по значениям 2,5% квантиля. Левая граница, которая отделяет 2.5 % данных для 1-го и 3-го регионов принимает отрицательное значение, что говорит о том, что более 2.5 % выборок имеют отрицательную прибыль. Согласно исследованию нужно оставить лишь те регионы, в которых вероятность убытков меньше 2,5%. Получается, что это 2 регион.
В связи с этим для дальнейшей работы рекомендую оставить только второй регион.
     </div>